In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
train.head()

In [ ]:
train.info()

In [ ]:
X=train.groupby('investment_id')
X.size().sort_values()


In [ ]:
X['time_id'].max().value_counts()


Choose investment_id=2140 as an example.

In [ ]:
X_2140=X.get_group(2140).sort_values('time_id')
X_2140

In [ ]:
X_2140=X_2140.drop(['row_id','time_id','investment_id'],axis=1).reset_index().drop('index',axis=1)
X_2140

Split Train&Test Dataset in order.

In [ ]:
X_2140_train=X_2140.iloc[:-120,:]
X_2140_test=X_2140.iloc[-120:,:]
print(len(X_2140_train),len(X_2140_test))

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler()
X_sca_train=scaler.fit_transform(X_2140_train.iloc[:,1:])
X_sca_test=scaler.transform(X_2140_test.iloc[:,1:])
print(len(X_sca_train),len(X_sca_test))

Start LSTM input preprocessing. Using first 5 periods features to predict the 5th target. 

In [ ]:
from collections import deque

In [ ]:
def create_input(dataset,memory_period):
    deq=deque(maxlen=memory_period)
    X=[]
    for i in dataset:
        deq.append(list(i))
        if len(deq)==memory_period:
            X.append(list(deq))
    return np.array(X)

X_sca_train_input=create_input(X_sca_train,5)
X_sca_test_input=create_input(X_sca_test,5)
print(X_sca_train_input.shape,X_sca_test_input.shape)

In [ ]:
Y_train=X_2140.iloc[4:-120,0].values
Y_test=X_2140.iloc[-116:,0].values
print(Y_train.shape,Y_test.shape)

In [ ]:
X_train=X_sca_train_input
X_test=X_sca_test_input
print(type(X_sca_train_input),type(Y_train))
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

In [ ]:
X_2140.isnull().sum().sum()

Start build NN.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
#X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)


In [ ]:
X_sca_train_input.shape[1:]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_model(optimizer):
    model=Sequential()
    model.add(LSTM(15,input_shape=X_sca_train_input.shape[1:],activation='relu',return_sequences=True))
    model.add(LSTM(15,activation='relu',return_sequences=True))
    model.add(LSTM(15,activation='relu'))
    model.add(Dense(15,activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam',loss='mse',metrics=['mape'])
    return model



In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
grid_model = KerasRegressor(build_fn=build_model,verbose=1,validation_data=(X_test,Y_test))

parameters = {'batch_size' : [16,20],
            'epochs' : [8,10],
            'optimizer' : ['adam','Adadelta'] }

grid_search = GridSearchCV(estimator = grid_model,
                          param_grid = parameters,
                          cv = 2)

In [ ]:
grid_search = grid_search.fit(X_train,Y_train)

In [ ]:

grid_search.best_params_

In [ ]:
my_model=grid_search.best_estimator_.model
my_model.summary()

In [ ]:
##model.fit(X_train,Y_train,batch_size=32,epochs=50,validation_data=(X_test,Y_test))

In [ ]:
from tensorflow.keras import metrics
